In [ ]:
import wrds
import pandas as pd
import time
import numpy as np
import sys

db = wrds.Connection(wrds_username = 'cmvj22')
# Require subscription to wrds. All Durham staff and students may request access via the University.


np.set_printoptions(edgeitems = 9, linewidth = 128,
                    formatter = {'float':lambda x: f'{x:.3f}'}, precision = 3, suppress = True)
pd.options.display.float_format = '{:.3f}'.format

#############################################################################
# SSH is faster than JupyterLab which is faster than normal Jupyter         #
#############################################################################
# On 2 cores on SSH Cloud, about 2/3rds take <15mins so run 2 or 3 and wait
# 15 mins for one to finish first. The rest take 60 mins, at random.

def get_sql(statement, date_col):
    res = db.raw_sql(statement)
    res[f'{date_col}'] = res[f'{date_col}'].apply(lambda x: pd.Timestamp(x))
    return res

def save_reopen_no_num(var, name, printed=True):
    var.to_csv(f'{name}.csv', index=False)
    if printed:
        print(f'{name} done at time: {time.strftime("%H:%M:%S", time.gmtime())}')
    return pd.read_csv(f'{name}.csv')#, parse_dates=[f'{date_col}'])

# Strikes are reported x1000 and contract is 100 shares
def contract_ids(capEforEuro, newYearDay, newYearEve):
    contracts_statement = f"""SELECT optionid, securityid, strike/100000 as strike, expiration, callput
                              FROM optionm.option_history
                              WHERE callput in ('C')
                                  AND exercisestyle = '{capEforEuro}'
                                  AND startdate BETWEEN '{newYearDay}' AND '{newYearEve}'
                              ORDER BY optionid ASC"""
    return get_sql(contracts_statement, 'expiration')

# Contracts are for 100 shares
def option_prices(year, contracts):
    all_prices_statement = f"""SELECT optionid, date AS date_traded, last/100 AS contract_price,
                                          underlyinglast/100 AS underlyings_price, volume AS contract_volume
                               FROM optionm.option_price_{year}
                               WHERE optionid in {tuple(contracts['optionid'])}
                                   AND volume > 2
                                   AND last > 5
                                   AND last < 40
                                   AND underlyinglast > 0.5
                                   AND specialsettlement = 0
                               GROUP BY optionid, date, volume, last, underlyinglast
                               ORDER BY optionid, date"""
    return get_sql(all_prices_statement, 'date_traded')

def combine_option_info(contracts, all_prices, combined):
    combined_1yr = pd.merge(contracts, all_prices, how = 'right', on = 'optionid')
    combined_1yr = combined_1yr.sort_values(by = ['optionid'], ignore_index = True)
    combined_1yr['days_to_maturity'] = (combined_1yr['expiration'] - combined_1yr['date_traded']) / np.timedelta64(1,'D')# This might be optimisable
    combined_1yr = combined_1yr[(1 < combined_1yr['days_to_maturity']) & (combined_1yr['days_to_maturity'] < 366)] # Only removes 3.5% of rows
    combined_1yr.drop(columns = ['expiration'], inplace = True)
    return pd.concat([combined, combined_1yr], axis = 0, ignore_index = True)

def toosmall(vec):
    if int(np.sum(vec)) == 0:
        print('Number too small - not enough quotes for splicing rates')
        print('Exiting script.')
        sys.exit()

print(f'Start time: {time.strftime("%H:%M:%S", time.gmtime())}')
start = time.time()

################################################################################
# Info on all trades for num option (Euro Call) contracts each year 2002-2019. #
################################################################################

combined      = pd.DataFrame()
contract_list = []
num_checked   =  pd.DataFrame()

for year in range(2002,2020):
    newYearDay = f'{year}-01-01'
    newYearEve = f'{year}-12-31'
    
    contracts = contract_ids('E', newYearDay, newYearEve)
    # A contract may be traded in the next calendar year so will check for last two years' optionids.
    contract_list.append(contracts)
    if year != 2002:
        contracts = pd.concat([contract_list.pop(0), contracts], axis = 0, ignore_index = True)
    num_checked = pd.concat([num_checked, contracts.optionid], axis = 0, ignore_index = True)
                            
    all_prices = option_prices(year, contracts)
    combined   = combine_option_info(contracts, all_prices, combined)
                            
combined.sort_values(by = ['securityid', 'callput', 'optionid', 'date_traded'], inplace = True, ignore_index = True)
num_checked = len(num_checked[0].unique())
# This ratio is sufficient for pricing so removes a feature for the network.
combined['moneyness'] = combined['underlyings_price'] / combined['strike']

#######################################################################################
# Adding interest rates (matching maturity and start date) for every options contract #
#######################################################################################

# Rates section below

# In addition to maturities outside of our (2,365) range, the unused maturities inside this range are
# collected as well to help make splicing more accurate and allow more date rows to be spliced.
interest_statement = f"""SELECT date AS date_traded, days AS days_to_maturity, rate
                         FROM optionm.zerocd
                         WHERE days BETWEEN 1 AND 373
                             AND date in {tuple(combined.date_traded.drop_duplicates().dt.strftime('%Y-%m-%d'))}
                         GROUP BY date, days, rate
                         ORDER BY date, days"""
interests = get_sql(interest_statement, 'date_traded')

# One row for every maturity per date including buffer and unused maturities to help splicing
repeated_dates = pd.DataFrame({'date_traded': np.repeat(interests.date_traded.unique(), 373)})
# Each date has one of every possible maturities
repeated_maturities = pd.DataFrame({ 'days_to_maturity': np.tile(np.linspace(1,373,373), len(interests.date_traded.unique()) ) })
# Every date has every maturity length
dated_maturities = pd.concat([repeated_dates, repeated_maturities], axis = 1)
# All known rate values for all date-maturity rows
all_rates = pd.merge(interests, dated_maturities, how = 'right', on = ['date_traded', 'days_to_maturity'])

# All available rates for all dates with enough rates to be spliced (most non-Fridays have at least 4 rates)
non_nan_counts = all_rates.groupby('date_traded')['rate'].apply( lambda vec: (len(vec)-np.isnan(vec).sum()) > 3 )
toosmall(non_nan_counts)
all_rates_purged = all_rates[np.repeat(non_nan_counts.values, 373)]

# Cubic-splice interpolating rates to all required maturities
def splicer(vec):
    return vec.interpolate(method = 'spline', order = 3, s = 0., limit_direction = 'both')
inter_rates = all_rates_purged.set_index('days_to_maturity')
inter_rates = pd.DataFrame(inter_rates.groupby('date_traded')['rate'].apply(splicer))
inter_rates.reset_index(level = 0, inplace = True)

req_rates_dated = pd.merge(all_rates_purged.reset_index(drop = True).drop(columns = ['rate']),
                           inter_rates, left_index = True, right_index = True)
req_rates_dated.drop(columns = ['days_to_maturity_x'], inplace = True)
req_rates_dated.rename(columns = {'days_to_maturity_y': 'days_to_maturity'}, inplace = True)

#Discard maturities not corresponding to option prices
req_rates_dated = req_rates_dated[req_rates_dated.days_to_maturity.isin(np.sort(combined.days_to_maturity.unique()))].reset_index(drop=True)

# Remove any dates where the rate points led to extreme rate values from inter/extrapolation
lim_min = min(interests.rate)
lim_max = max(interests.rate)
well_spliced_dates = req_rates_dated.groupby('date_traded', as_index = False)['rate'].apply(
                        lambda vec: (lim_min < min(vec)) & (max(vec) < lim_max) ).drop(columns = ['date_traded'])
toosmall(well_spliced_dates)
req_rates_dated = req_rates_dated[well_spliced_dates.rate.repeat(len(combined.days_to_maturity.unique())).reset_index(drop=True)]
# Rates were reported as percentages
req_rates_dated['rate'] = req_rates_dated['rate']/100

####################################################################
# Join 'combined' and 'req_rates_dated' for a single complete table#
####################################################################

data = pd.merge(combined, req_rates_dated, on = ['date_traded', 'days_to_maturity'])

# Company mergers lead to existing contracts being quoted for both securityid's.
data.drop_duplicates(subset = data.columns.difference(['securityid']), ignore_index = True, inplace = True)
# Some repetition of quotes where only difference is strike being slightly changed.
data.drop_duplicates(subset = data.columns.difference(['strike']), ignore_index = True, inplace = True)
# Calculation of days_to_maturity has two answers, typically one day off.
data.drop_duplicates(subset = data.columns.difference(['days_to_maturity', 'rate']), ignore_index = True, inplace = True)
# Some calls are duplicated -- Not sure why.
data.drop_duplicates(subset = data.columns.difference(['optionid']), ignore_index = True, inplace = True)
# Some quotes duplicated across exchanges.
data.drop_duplicates(subset = data.columns.difference(['optionid', 'contract_volume']), ignore_index = True, inplace = True)


# Remove any companys with dividends or stock splits (62% of companies but 1.4% of quotes including puts)
data = data[~data['securityid'].isin(db.get_table('optionm', 'distribution').securityid.unique())]
# There are some outlier contracts which are either accidental trades or the strike/quote multiplier was missed
data = data[(0.5 < data['moneyness']) & (data['moneyness'] < 1.5)]

###############################################################################
# Adding historical realised volatility for 30 days prior to each data_traded #
###############################################################################

vol_statement = f"""SELECT securityid, date AS date_traded, volatility
                    FROM optionm.historical_volatility
                    WHERE days = 30
                        AND securityid in {tuple(data.securityid.unique())}
                        AND date in {tuple(data.date_traded.drop_duplicates().dt.strftime('%Y-%m-%d'))}
                    GROUP BY securityid, date, volatility
                    ORDER BY securityid, date"""
vols = get_sql(vol_statement, 'date_traded')
data = pd.merge(data, vols, how = 'left').dropna() # A small handful of dates don't have data, hence .dropna()
data = data[data.volatility < 2] # Market stress causes severe outliers for the Black-Scholes accuracy benchmark

###############################################
#                 # Summary #                 #
###############################################

num_contracts = len(data.optionid.unique())
num_quotes    = len(data.index)
num_calls     = len(data[data['callput'] == 'C'])
num_puts      = len(data.index) - num_calls

data = data[['strike', 'moneyness', 'rate', 'volatility', 'days_to_maturity', 'contract_price']]
data = save_reopen_no_num(data, f'calls_OMrates{len(data.index)}')


print(f'End time: {time.strftime("%H:%M:%S", time.localtime())}')
print(f'Time taken: {time.time() - start:.1f}')

if True: print(f"""

This script uses the OptionMetrics interest rates.
These are inconsistent with put-call parity so only
real calls are returned. Another script uses
put-call parity to deduce the rate.

For 2002 to 2019 inclusive, this database has
3,301,638 positive-volume quotes on
1,267,300 distinct Euro call contracts.

This script checked {num_checked:,} Euro-call contracts
and after removing unsuitable quotes, we returned
{num_quotes:,} quotes from {num_contracts:,} contracts;
{100*num_quotes/(3301638):,.1f}% and {100*num_contracts/(1267300):,.1f}% of quotes and contracts respectively.
All are converted to call prices in the output file.

Two-thirds of quotes are unfortunately filtered out
because the Black-Scholes accuracy benchmark performs
exceptionally poorly for quotes with very large volatility
or prices near the ticksize so these are removed as
the error appears systematic (extreme market stress on
particular days rather than the volatility smile itself).

Also, this database is truly enormous: there are
1,096,159,594 total quotes for 2002-2019,
107,988,528 of which are for Euro calls.""")


db.close()